# SQLAlchemy ORM

In [1]:
from sqlalchemy import create_engine, ForeignKey, Table, Column
from sqlalchemy import Integer, String, Numeric, Date, DateTime
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base
from datetime import date, datetime

In [2]:
Base = declarative_base()

In [3]:
class Author(Base):
    __tablename__ = 'authors'
    id = Column(Integer, primary_key=True)
    name = Column(String, unique=True)
    birthday = Column(Date)
    
    def __init__(self, name, birthday):
        self.name = name
        self.birthday = birthday

In [4]:
class Book(Base):
    __tablename__ = 'books'
    id = Column(Integer, primary_key=True)
    title = Column(String, unique=True)
    published_datetime = Column(DateTime)
    author_id = Column(Integer, ForeignKey('authors.id')) # 1:many relationship
    author = relationship("Author", backref='books')
    
    def __init__(self, title, author, published_datetime):
        self.title = title
        self.author = author
        self.published_datetime = published_datetime

In [5]:
book_store_mapping = Table('books_stores', Base.metadata,
                           Column('book_id', Integer, ForeignKey('books.id')),
                           Column('store_id', Integer, ForeignKey('stores.id')))

class Store(Base):
    __tablename__ = 'stores'
    id = Column(Integer, primary_key=True)
    name = Column(String, unique=True)
    books = relationship("Book", secondary=book_store_mapping)
    
    def __init__(self, name):
        self.name = name

In [6]:
alan_smithee = Author('Alan Smithee', date(1978, 4, 17))
jane_doe = Author('Jane Doe', date(1984, 2, 28))

In [7]:
python_tutorial = Book('Python Tutorial', alan_smithee, datetime.utcnow())
guide = Book("Hitchhiker's Guide to Python", jane_doe, 
             datetime.fromisoformat('2019-01-23T14:34:21'))
almanach = Book('Almanach of Python Wisdom', jane_doe, datetime(2019,1,23,14,14,21))

In [8]:
inn = Store("Bookkeeper's Inn")
library = Store('Great Library')

In [9]:
inn.books = [python_tutorial, almanach]
library.books = [guide, python_tutorial, almanach]

In [10]:
con_str = 'sqlite://' # SQLite database in memory
engine = create_engine(con_str)
Session = sessionmaker(bind=engine)

In [11]:
Base.metadata.create_all(engine)

In [12]:
session = Session()

In [13]:
session.add_all((python_tutorial, guide, almanach, inn))
session.add(library)

In [14]:
session.commit()
session.close()